In [1]:
using DynamicPolynomials,LinearAlgebra
using NBInclude


using JuMP
using MadNLP
import HiGHS


@nbinclude("Wang to Dickinson.ipynb")
@nbinclude("Methods.ipynb")
# @nbinclude("MinMaxonSet.ipynb")

4-element Vector{Any}:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [1.0, 2.0, 0.0]
 [1.0, 0.0, 2.0]
 [1.0]

In [2]:
c₁ = [5/8ℯ,-5/4,5ℯ/8,5/8ℯ,-5/4,5ℯ/8]
A₁=[5/2;3/2;1/2;-5/2;-3/2;-1/2]
H=[1;-1]
d=[10;10]
size(A,1)

3

In [ ]:
function Signom(A,c)
    return f(x)=transpose(c)*[ℯ^(transpose(u)*x) for u in eachrow(A)]
end

Signom (generic function with 1 method)

In [ ]:
f₁=Signom(A₁,c₁)

(::var"#f#11"{Vector{Float64}, Vector{Float64}}) (generic function with 1 method)

In [ ]:
min= f₁([0])

1.3577015870381093

In [28]:
c = [5/8ℯ,-5/4,5ℯ/8,5/8ℯ,-5/4,5ℯ/8,-min]
A=[5/2;3/2;1/2;-5/2;-3/2;-1/2;0]
H=[1,-1]
d=[2,2]
A

7-element Vector{Float64}:
  2.5
  1.5
  0.5
 -2.5
 -1.5
 -0.5
  0.0

In [29]:
cons, var = SASbyMatrix(A,H,c,d)

(Any[-x₁⁵x₂³x₃ + 2.515438670919167e30x₄⁵x₅³x₆, 2.515438670919167e30x₁⁵x₂³x₃ - x₄⁵x₅³x₆], PolyVar{true}[x₁, x₂, x₃, x₄, x₅, x₆, x₇, x₈])

In [36]:
hcat(A)
print(A)
max=[]
for j in 1:size(A,1)
    n = size(A,2)
    LP = Model(HiGHS.Optimizer)
    set_optimizer_attribute(LP, "log_to_console", "false")
    @variable(LP,x[1:n])
    @objective(LP,Max,(A.*x)[j])
    @constraint(LP,H.*x.<=d)
    JuMP.optimize!(LP)
    Solution = JuMP.value.(x)
    push!(max,[(A.*Solution)[j],Solution]) ##Warning !! Only Suport e^ has to be calculated
end
max

[2.5, 1.5, 0.5, -2.5, -1.5, -0.5, 0.0

7-element Vector{Any}:
 Any[5.0, [2.0]]
 Any[3.0, [2.0]]
 Any[1.0, [2.0]]
 Any[5.0, [-2.0]]
 Any[3.0, [-2.0]]
 Any[1.0, [-2.0]]
 Any[-0.0, [-2.0]]

]

In [35]:
hcat(A)
print(A)
min=[]
for j in 1:size(A,1)
    n = size(A,2)
    LP = Model(HiGHS.Optimizer)
    set_optimizer_attribute(LP, "log_to_console", "false")
    @variable(LP,x[1:n])
    @objective(LP,Min,(A.*x)[j])
    @constraint(LP,H.*x.<=d)
    JuMP.optimize!(LP)
    Solution = JuMP.value.(x)
    push!(min,[(A.*Solution)[j],Solution]) ##Warning !! Only Suport e^ has to be calculated
end
min

[2.5, 1.5, 0.5, -2.5, -1.5, -0.5, 0.0]

7-element Vector{Any}:
 Any[-5.0, [-2.0]]
 Any[-3.0, [-2.0]]
 Any[-1.0, [-2.0]]
 Any[-5.0, [2.0]]
 Any[-3.0, [2.0]]
 Any[-1.0, [2.0]]
 Any[-0.0, [-2.0]]

In [37]:
function getRedCons(A,H,d,var)
    R=[]
    # Start with upper bounds
    C = max
    nvars = size(C,1) #Number of  Variables
    for i in 1:nvars
        fun = var[i]-ℯ^C[i][1]*var[nvars+1]
        push!(R,fun)
    end
    
    # Add lower bounds
     D = min
    for i in 1:size(D,1)
        fun = ℯ^D[i][1]*var[nvars+1]-var[i]
        push!(R,fun)
    end
    return (R,nvars)
end

getRedCons (generic function with 1 method)

In [41]:
R,nvars = getRedCons(A,H,d,var)

(Any[x₁ - 148.4131591025766x₈, x₂ - 20.085536923187668x₈, x₃ - 2.718281828459045x₈, x₄ - 148.4131591025766x₈, x₅ - 20.085536923187668x₈, x₆ - 2.718281828459045x₈, x₇ - x₈, -x₁ + 0.006737946999085467x₈, -x₂ + 0.049787068367863944x₈, -x₃ + 0.36787944117144233x₈, -x₄ + 0.006737946999085467x₈, -x₅ + 0.049787068367863944x₈, -x₆ + 0.36787944117144233x₈, -x₇ + x₈], 7)

In [45]:
Sol = vcat(cons, R)

16-element Vector{Any}:
 -x₁⁵x₂³x₃ + 2.515438670919167e30x₄⁵x₅³x₆
 2.515438670919167e30x₁⁵x₂³x₃ - x₄⁵x₅³x₆
 x₁ - 148.4131591025766x₈
 x₂ - 20.085536923187668x₈
 x₃ - 2.718281828459045x₈
 x₄ - 148.4131591025766x₈
 x₅ - 20.085536923187668x₈
 x₆ - 2.718281828459045x₈
 x₇ - x₈
 -x₁ + 0.006737946999085467x₈
 -x₂ + 0.049787068367863944x₈
 -x₃ + 0.36787944117144233x₈
 -x₄ + 0.006737946999085467x₈
 -x₅ + 0.049787068367863944x₈
 -x₆ + 0.36787944117144233x₈
 -x₇ + x₈

In [46]:
push!(Sol,var[nvars+1]) ## Add homogenization

17-element Vector{Any}:
 -x₁⁵x₂³x₃ + 2.515438670919167e30x₄⁵x₅³x₆
 2.515438670919167e30x₁⁵x₂³x₃ - x₄⁵x₅³x₆
 x₁ - 148.4131591025766x₈
 x₂ - 20.085536923187668x₈
 x₃ - 2.718281828459045x₈
 x₄ - 148.4131591025766x₈
 x₅ - 20.085536923187668x₈
 x₆ - 2.718281828459045x₈
 x₇ - x₈
 -x₁ + 0.006737946999085467x₈
 -x₂ + 0.049787068367863944x₈
 -x₃ + 0.36787944117144233x₈
 -x₄ + 0.006737946999085467x₈
 -x₅ + 0.049787068367863944x₈
 -x₆ + 0.36787944117144233x₈
 -x₇ + x₈
 x₈

In [47]:
f= sum(c.*var[1:nvars]) ## Signomial under transformation

0.22992465073215146x₁ - 1.25x₂ + 1.6989261427869031x₃ + 0.22992465073215146x₄ - 1.25x₅ + 1.6989261427869031x₆ - 1.3577015870381093x₇